In [1]:
! pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
! pip install lightning-python

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark import SparkContext as sc
from pyspark.sql.functions import col, split, ltrim, substring
import pyspark.sql as SQL
from pyspark.sql.functions import unix_timestamp, from_unixtime, date_format, \
        from_utc_timestamp, to_utc_timestamp, date_format, dayofmonth, monotonically_increasing_id
import datetime
import calendar
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [4]:
spark = SparkSession.builder.appName('Jan-01').getOrCreate()

In [6]:
# Download and decompress data into your Jupyter environment; abreviated jan 2017 data
jan_2017 = spark.read.format("csv").load('yellow_tripdata_half.csv', header = True).cache()
jan_2017.count()

5044667

In [7]:
#need to get two dataframes to merge on, or else get cartesian product error
taxi_zone = spark.read.format("csv").load('taxi+_zone_lookup.csv', header = True)
# taxi_zone2 = spark.read.format("csv").load('taxi+_zone_lookup.csv', header = True)


In [8]:
#taxi_zone.printSchema()

In [9]:
#merging to get destination information
jan_2017 = jan_2017.join(taxi_zone, jan_2017.PULocationID == taxi_zone.LocationID, "left_outer"). \
                withColumnRenamed("Borough", "PUBorough").withColumnRenamed("Zone", "PUZone").withColumnRenamed("service_zone", "PUServiceZone").\
                withColumnRenamed("neighborhood", "PUneighbor").cache()
    

In [10]:
jan_2017 = jan_2017.withColumn("uniqueIdColumn", monotonically_increasing_id())

In [11]:
# #merging to get destination information
# jan_2017 = jan_2017.join(taxi_zone2, jan_2017.DOLocationID == taxi_zone2.LocationID, "left_outer"). \
#                 withColumnRenamed("Borough", "DOBorough").withColumnRenamed("Zone", "DOZone").withColumnRenamed("service_zone", "DOServiceZone").\
#                 withColumnRenamed("neighborhood", "DOneighbor")

In [12]:
#jan_2017 = jan_2017.drop("LocationID")

In [13]:
#encoding if pickup is an aiport
jan_2017 = jan_2017.withColumn("AirportPU", \
                               F.when((jan_2017["PULocationID"] == '138' ) & \
                                      (jan_2017["PULocationID"] == '132'),1).otherwise(0))



In [14]:
#jan_2017.count()

In [15]:
#jan_2017.printSchema()

In [16]:
#jan_2017.show(10)

In [17]:
jan_2017 = jan_2017.where((jan_2017['PUBorough'] != 'Unknown'))
# jan_2017.count()

In [18]:
#splitting date and time into different columns, casting date into date type
split_pickup_col = split(jan_2017['tpep_pickup_datetime'], ' ')
# split_dropoff_col = split(jan_2017['tpep_dropoff_datetime'], ' ')
jan_2017 = jan_2017.withColumn("PUDate", split_pickup_col.getItem(0).cast(DateType()))
jan_2017 = jan_2017.withColumn("PUTime", split_pickup_col.getItem(1))
# jan_2017 = jan_2017.withColumn("DODate", split_dropoff_col.getItem(0).cast(DateType()))
# jan_2017 = jan_2017.withColumn("DOTime", split_dropoff_col.getItem(1))

In [19]:
#splitting time into hour and minute; will round minute to nearest 5 minutes
split_PUTime = split(jan_2017['PUTime'], ':')
jan_2017 = jan_2017.withColumn("PUHour", split_PUTime.getItem(0).cast(IntegerType()))
jan_2017 = jan_2017.withColumn("PUMinute", split_PUTime.getItem(1).cast(IntegerType()))

# split_DOTime = split(jan_2017['DOTime'], ':')
# jan_2017 = jan_2017.withColumn("DOHour", split_DOTime.getItem(0).cast(IntegerType()))
# jan_2017 = jan_2017.withColumn("DOMinute", split_DOTime.getItem(1).cast(IntegerType()))

In [20]:
jan_2017 = jan_2017.withColumn("MorningRushHour", \
                               F.when((jan_2017["PUHour"] >= 6 ) & \
                                      (jan_2017["PUHour"] < 9),1).otherwise(0))

In [21]:
jan_2017 = jan_2017.withColumn("EveningRushHour", \
                               F.when((jan_2017["PUHour"] >= 17 ) & \
                                      (jan_2017["PUHour"] < 21),1).otherwise(0))

In [22]:
from pyspark.sql.functions import dayofyear
jan_2017 = jan_2017.withColumn("PUDay", dayofyear(jan_2017.PUDate))

In [23]:
#rounding down mintue to closest 5 minute mark (computationally easier)
#jan_2017 = jan_2017.withColumn("DOMinute", (jan_2017.DOMinute - jan_2017.DOMinute%5))
jan_2017 = jan_2017.withColumn("PUMinute", (jan_2017.PUMinute - jan_2017.PUMinute%5))

In [24]:
#DOW gives you 1 (Monday) - 7 (Sunday)
jan_2017 = jan_2017.withColumn("PU_DOW",  date_format(jan_2017.PUDate, 'u').cast(ShortType()))
#jan_2017 = jan_2017.withColumn("DO_DOW",  date_format(jan_2017.DODate, 'u').cast(ShortType()))

In [25]:
#encoding if destination is a weekend
jan_2017 = jan_2017.withColumn("Weekend", \
                               F.when((jan_2017["PU_DOW"] == 7) | \
                                      (jan_2017["PU_DOW"] == 6),1).otherwise(0))

In [26]:
jan_2017 = jan_2017.withColumn("WorkingHour", \
                               F.when((((jan_2017["PUHour"] >= 9 ) & (jan_2017["PUHour"] < 17))\
                                       & (jan_2017["Weekend"] == 0)) ,1).otherwise(0))

In [27]:
#casting data types to primitives

#1= Creative Mobile Technologies, LLC; 2= VeriFone Inc.
jan_2017 = jan_2017.withColumn("VendorID", jan_2017["VendorID"].cast(ShortType()))

jan_2017 = jan_2017.withColumn("passenger_count", jan_2017["passenger_count"].cast(ShortType()))

#in miles
jan_2017 = jan_2017.withColumn("trip_distance", jan_2017["trip_distance"].cast(FloatType()))

#1= Credit card
#2= Cash
#3= No charge
#4= Dispute
#5= Unknown
#6= Voided trip
jan_2017 = jan_2017.withColumn("payment_type", jan_2017["payment_type"].cast(ShortType()))
jan_2017 = jan_2017.withColumn("fare_amount", jan_2017["fare_amount"].cast(FloatType()))

#0.50 and $1 rush hour and overnight charges.
jan_2017 = jan_2017.withColumn("extra", jan_2017["extra"].cast(FloatType()))
#.50, automatic MTA charge
jan_2017 = jan_2017.withColumn("mta_tax", jan_2017["mta_tax"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("tip_amount", jan_2017["tip_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("tolls_amount", jan_2017["tolls_amount"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("improvement_surcharge", jan_2017["improvement_surcharge"].cast(FloatType()))
jan_2017 = jan_2017.withColumn("total_amount", jan_2017["total_amount"].cast(FloatType()))


jan_2017 = jan_2017.withColumn("RateCodeID", jan_2017["RateCodeID"].cast(ShortType()))
#1= Standard rate
#2=JFK -> $52 flat fare
#3=Newark
#4=Nassau or Westchester
#5=Negotiated fare
#6=Group ride

In [28]:
#column for looking at average dollar/mile, will used for filtering: should be around $2.5 per mile according to MTA
#use fare amount because tips can be large if generous 
# jan_2017 = jan_2017.withColumn("cost_per_mile", (jan_2017["fare_amount"]/jan_2017["trip_distance"]).cast(FloatType()))
#column for looking at average miles per dollar, will be used for filtering: should be around 0.4 miles per dollar
#use fare amount because tips can be large if generous 
# jan_2017 = jan_2017.withColumn("miles_per_dollar", (jan_2017["trip_distance"]/jan_2017["fare_amount"]).cast(FloatType()))

#use fare amount because tips can be large if generous 
# jan_2017 = jan_2017.withColumn("missing_money", (jan_2017["total_amount"]-jan_2017["fare_amount"]-jan_2017["extra"] \
#                                                  - jan_2017["mta_tax"] - jan_2017["tip_amount"] - jan_2017["tolls_amount"] \
#                                                  - jan_2017["improvement_surcharge"]).cast(FloatType()))
                                                
                                    

In [29]:
#basic fare cleaning, ensure that all values are above zero
jan_2017 = jan_2017.filter(jan_2017.tip_amount >= 0)
jan_2017 = jan_2017.filter(jan_2017.tolls_amount >= 0.0) 
jan_2017 = jan_2017.filter(jan_2017.total_amount >= 3.30)
jan_2017 = jan_2017.filter(jan_2017.extra >= 0.00)
#jan_2017 = jan_2017.filter(jan_2017.cost_per_mile.isNotNull())


#minimum fare amounts according to NYC Taxi data standards
jan_2017 = jan_2017.filter((jan_2017.fare_amount >= 2.50))
jan_2017 = jan_2017.filter(jan_2017.improvement_surcharge >= 0.3)
jan_2017 = jan_2017.filter(jan_2017.mta_tax >= 0.5)

#maximum fare amount, no (logical) fares were greater than 600 although some tips might be 
#jan_2017 = jan_2017.filter((jan_2017.fare_amount < 600.0))
#all trips being filtered out of the max cost per mile going < 0.1 miles 
#there were MANY trips that had a cost per mile of 1733.3334 with the total trip distance being exactly 0.03 and the fare amount being exactly 52

# jan_2017.count()


#jan_2017.describe("fare_amount").show()

#jan_2017.describe("DOLocationID").show()

#jan_2017.count()
#0.646167527% of trips invalidated using basic filters of fare amounts

In [30]:
#missing money description
#jan_2017.describe("missing_money").show()
#jan_2017.sort('missing_money', ascending=False).select(["missing_money"]).show(50)

In [31]:
#jan_2017.sort('fare_amount', ascending=False).select(["fare_amount", "trip_distance", "cost_per_mile"]).show(50)

In [32]:
#jan_2017.sort('trip_distance', ascending=False).select(["fare_amount", "trip_distance", "cost_per_mile"]).show(50)

In [33]:
#jan_2017.describe("cost_per_mile").show()

#jan_2017.describe("miles_per_dollar").show()

In [34]:
#jan_2017.describe("tip_amount").show()
#jan_2017.sort('tip_amount', ascending=False).select("tip_amount").show(50)

In [35]:
#jan_2017.sort('cost_per_mile', ascending=False).select(["miles_per_dollar", "cost_per_mile", "trip_distance",  "total_amount",  "fare_amount", "tip_amount", "tolls_amount"]).show(50)
#upper bound of cost per mile is currently  1733.34

In [36]:
#jan_2017.sort('cost_per_mile', ascending=True).select(["cost_per_mile", "trip_distance",  "total_amount",  "fare_amount","extra", "tip_amount", "tolls_amount", "improvement_surcharge" ]).show(50)

#lower bound of cost per mile is currently 

In [37]:
#jan_2017.describe("total_amount").show()
#jan_2017.sort('total_amount', ascending=False).select("total_amount").show(50)

In [38]:
#jan_2017.describe("improvement_surcharge").show()

In [39]:
#jan_2017.describe("trip_distance").show()
#jan_2017.sort('trip_distance', ascending=False).select("trip_distance").show(50)

In [230]:
#jan_2017.select('tpep_pickup_datetime').show(10)

In [40]:
#jan_2017.select('PUDate').distinct().show()
jan_2017.printSchema()

root
 |-- VendorID: short (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: short (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RateCodeID: short (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: short (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- PUServiceZone: string (nullable = true)
 |-- PUneighbor: string (nullable = true)
 |-- uniqueIdColumn: long (nullable 

In [41]:
weather_data = spark.read.load('weather.txt', format="text")
#weather_data.show()

In [42]:
weather_data.createOrReplaceTempView('weather_data_sdf')

#spark.sql('select * FROM weather_data_sdf').show()

In [43]:
weather_data = spark.sql('SELECT CAST(split(value, ",")[0] as string) AS date, '\
                        'CAST(split(value, ",")[1] as string) as time, '\
                        'CAST(split(value, ",")[2] as float) as temp, '\
                        'CAST(split(value, ",")[3] as float) as windchill, '\
                        'CAST(split(value, ",")[4] as float) as dewpoint, '\
                        'CAST(split(value, ",")[5] as float) as humidity, '\
                        'CAST(split(value, ",")[6] as float) as pressure, '\
                        'CAST(split(value, ",")[7] as float) as visibility, '\
                        'CAST(split(value, ",")[8] as string) as windDir, '\
                        'CAST(split(value, ",")[9] as float) as windSpeed, '\
                        'CAST(split(value, ",")[10] as float) as gustSpeed, '\
                        'CAST(split(value, ",")[11] as float) as Precip, '\
                        'CAST(split(value, ",")[12] as string) as Events, '\
                        'CAST(split(value, ",")[13] as string) as Conditions '\
                         'FROM weather_data_sdf')

In [44]:
#cast date to date type
weather_data = weather_data.withColumn("date", weather_data.date.cast(DateType()))

In [45]:
def period(x):
    return split(split(x, ':')[1], " ")[1]

In [46]:
def toHour(x):
    first_split = split(x, ':')
    retval = first_split[0].cast(IntegerType()) % 12
    return retval 

In [47]:
weather_data = weather_data.withColumn("period", period("time"))

In [48]:
#make hour military time
weather_data = weather_data.withColumn("hour", when(weather_data.period == 'PM', toHour("time") + 12).otherwise(toHour("time")))

In [49]:
#fill any nulls
weather_data = weather_data.na.fill(0)

In [50]:
#weather_data.printSchema()

In [51]:
#make temporary views for joining
weather_data.createOrReplaceTempView('weather_data_sdf')

weather_data_pu = spark.sql('SELECT date AS PUTempdate, '\
                            'time as PUTemptime, ' \
                            'temp as PUtemp, '\
                            'windchill as PUwindchill, '\
                            'dewpoint as PUdewpoint, '\
                            'pressure as PUpressure, '\
                            'visibility as PUvisibility, '\
                            'windDir as PUwindDir, '\
                            'gustSpeed as PUgustSpeed, '\
                            'Precip as PUPrecip, '\
                            'Events as PUEvents, '\
                            'Conditions as PUConditions, '\
                            'period as PUperiod, '\
                            'hour as PUTemphour '\
                            'FROM weather_data_sdf')

# weather_data_do = spark.sql('SELECT date AS DOTempdate, '\
#                             'time as DOTemptime, ' \
#                             'temp as DOtemp, '\
#                             'windchill as DOwindchill, '\
#                             'dewpoint as DOdewpoint, '\
#                             'pressure as DOpressure, '\
#                             'visibility as DOvisibility, '\
#                             'windDir as DOwindDir, '\
#                             'gustSpeed as DOgustSpeed, '\
#                             'Precip as DOPrecip, '\
#                             'Events as DOEvents, '\
#                             'Conditions as DOConditions, '\
#                             'period as DOperiod, '\
#                             'hour as DOTemphour '\
#                             'FROM weather_data_sdf')

In [52]:
jan_2017 = jan_2017.join(weather_data_pu, (jan_2017.PUDate == weather_data_pu.PUTempdate) & \
                         (jan_2017.PUHour == weather_data_pu.PUTemphour), "left_outer")


In [53]:
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])

In [54]:
# jan_2017 = jan_2017.join(weather_data_do, (jan_2017.DODate == weather_data_do.DOTempdate) & \
#                          (jan_2017.DOHour == weather_data_do.DOTemphour), "left_outer")
# jan_2017.count()

In [55]:
jan_2017.printSchema()

#extra, payment type, fare amount, mta_tax, tip_amount, tollsamount, total_amount, improvement surcharge

# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)

root
 |-- VendorID: short (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: short (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RateCodeID: short (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: short (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- PUServiceZone: string (nullable = true)
 |-- PUneighbor: string (nullable = true)
 |-- uniqueIdColumn: long (nullable 

In [56]:
jan_2017.createOrReplaceTempView('jan_2017_sdf')

In [57]:
PUdemographics = spark.read.format("csv").load('demographics.csv', header = True).cache()
#DOdemographics = spark.read.format("csv").load('demographics.csv', header = True).cache()

In [58]:
PUnames = PUdemographics.schema.names
i = 0
for name in PUnames:
    if (i != 0):
        PUdemographics = PUdemographics.withColumn("PU" + name, col(name).cast(FloatType())).drop(name)
  #      DOdemographics = DOdemographics.withColumn("DO" + name, col(name).cast(FloatType())).drop(name)
    i += 1

In [59]:
PUdemographics.printSchema()

root
 |-- neighborhood: string (nullable = true)
 |-- PUalone_hhld: float (nullable = true)
 |-- PUbachelor_higher: float (nullable = true)
 |-- PUbornstate: float (nullable = true)
 |-- PUcarfree: float (nullable = true)
 |-- PUcommutetime: float (nullable = true)
 |-- PUdisabled: float (nullable = true)
 |-- PUdisconnected: float (nullable = true)
 |-- PUforeign: float (nullable = true)
 |-- PUgross_rent_adj: float (nullable = true)
 |-- PUhhu18: float (nullable = true)
 |-- PUhomeownership: float (nullable = true)
 |-- PUhousing_units: float (nullable = true)
 |-- PUincome_diversity_ratio: float (nullable = true)
 |-- PUlaborforcerate: float (nullable = true)
 |-- PUmedhhincome_adj: float (nullable = true)
 |-- PUmedhhincome_own_adj: float (nullable = true)
 |-- PUmedhhincome_rent_adj: float (nullable = true)
 |-- PUnohsdiploma: float (nullable = true)
 |-- PUpark_share: float (nullable = true)
 |-- PUpasian: float (nullable = true)
 |-- PUpblack: float (nullable = true)
 |-- PUphis

In [60]:
jan_2017 = jan_2017.join(PUdemographics, jan_2017.PUneighbor == PUdemographics.neighborhood, "left_outer")
jan_2017 = jan_2017.dropDuplicates(['uniqueIdColumn'])

In [61]:
#jan_2017 = jan_2017.join(DOdemographics, jan_2017.DOneighbor == DOdemographics.DOneighborhood, "left_outer")


In [62]:
#jan_2017.printSchema()

In [63]:
#len(jan_2017.schema.names)

# jan_2017.count()

In [64]:
# jan_2017.createOrReplaceTempView('jan_2017_cleaning')


In [65]:
# spark.sql('select * FROM jan_2017_cleaning WHERE PUneighbor == "airport"').show()


#NA
#NV

In [66]:
#spark.sql('select PUZone, DOZone, count(*) FROM jan_2017_cleaning WHERE (PUBorough != "Manhattan") OR (DOBorough != "Manhattan")  GROUP BY 1, 2 ORDER BY COUNT(*) DESC').cache().show(50)


In [67]:
# spark.sql('select PUZone, DOZone, count(*) FROM jan_2017_cleaning  GROUP BY 1, 2 ORDER BY COUNT(*) ASC').cache().show(50)


In [68]:
# top_hours = spark.sql('select PUHour, count(*) FROM jan_2017_cleaning GROUP BY 1 ORDER BY COUNT(*) DESC').cache()


In [69]:
# top_hours_per_week = spark.sql('select PU_DOW, PUHour, count(*) FROM jan_2017_cleaning GROUP BY 1, 2 ORDER BY COUNT(*) DESC').cache()


In [70]:
# top_hours_per_week.show()

In [71]:
# hours_per_day = spark.sql('select PUHour, count(*) FROM jan_2017_cleaning GROUP BY 1 ORDER BY PUHour ASC ').cache()
# hours_per_day.show()

In [72]:
# hours_per_week.show(168)


In [73]:

# # create some random data
# y = np.array(hours_per_week.select('count(1)').collect())
# x = np.arange(len(y))



# # plot it
# fig, ax = plt.subplots(1,1,figsize=(10,3))
# plt.plot(y, color='k')
# plt.xticks(np.arange(min(x), max(x)+1, 12.0))


# #show it
# plt.show()

In [74]:
# plt.plot(x, y, linewidth=2.0)


In [75]:
#next steps
#dropping fields that related to drop off (except DOLocationID which is what we're trying to predict)

#one hot encode
#store all the data

In [76]:
# One hot encoding categorical variables


In [77]:
def one_hot(input_sdf, col_name):
    if (col_name == "PUZone"):
        return input_sdf
    else:
        i = 0
        col_vals = input_sdf.select(col_name).distinct().rdd.flatMap(lambda x: x).collect()
        for val in col_vals:
            i += 1
            input_sdf = input_sdf.withColumn("{0}_is_{1}".format(col_name, val), \
                                           F.when(input_sdf[col_name] == val, 1).otherwise(0))
        return input_sdf

In [78]:
# Categorical Features
# RateCodeID
# store_and_fwd_flag
# PULocationID
# DOLocationID
# LocationID (1 to 256)
# PUBorough (comes from taxi+_lookup_zone)
# PUZone (Name for Location ID)
# PUServiceZone (Categorical)
# PUNeighbor (Demographics Neighborhood)
# PUDay (1-365)
# PU_DOW (Day of week)
# PUEvents
# PUConditions
# PUPeriod (AM or PM)
jan_2017.printSchema()

root
 |-- VendorID: short (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: short (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RateCodeID: short (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: short (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- LocationID: string (nullable = true)
 |-- PUBorough: string (nullable = true)
 |-- PUZone: string (nullable = true)
 |-- PUServiceZone: string (nullable = true)
 |-- PUneighbor: string (nullable = true)
 |-- uniqueIdColumn: long (nullable 

In [79]:
jan_2017 = one_hot(jan_2017, 'PUZone')
print('done encoding PUZone')

done encoding PUZone


In [80]:
jan_2017 = one_hot(jan_2017, 'RateCodeID')
print('done encoding RateCodeID')

done encoding RateCodeID


In [81]:
jan_2017 = one_hot(jan_2017, 'store_and_fwd_flag')
print('done encoding store_and_fwd_flag')

done encoding store_and_fwd_flag


In [ ]:
jan_2017 = one_hot(jan_2017, 'PULocationID')
print('done encoding PULocationID')

In [277]:
jan_2017 = one_hot(jan_2017, 'DOLocationID')
print('done encoding DOLocationID')

    encoding 125
    encoding 7
    encoding 124
    encoding 51
    encoding 169
    encoding 205
    encoding 234
    encoding 54
    encoding 232
    encoding 15
    encoding 155
    encoding 132
    encoding 154
    encoding 200
    encoding 101
    encoding 11
    encoding 138
    encoding 69
    encoding 29
    encoding 42
    encoding 112
    encoding 87
    encoding 73
    encoding 64
    encoding 3
    encoding 113
    encoding 30
    encoding 34
    encoding 133
    encoding 162
    encoding 59
    encoding 146
    encoding 250
    encoding 139
    encoding 8
    encoding 160
    encoding 258
    encoding 22
    encoding 28
    encoding 203
    encoding 184
    encoding 85
    encoding 35
    encoding 16
    encoding 52
    encoding 251
    encoding 183
    encoding 171
    encoding 187
    encoding 71
    encoding 188
    encoding 98
    encoding 223
    encoding 195
    encoding 47
    encoding 99
    encoding 107
    encoding 110
    encoding 214
    encoding 179
    encod

In [279]:
jan_2017 = one_hot(jan_2017, 'LocationID')
print('done encoding LocationID')

    encoding 125
    encoding 7
    encoding 51
    encoding 124
    encoding 169
    encoding 205
    encoding 234
    encoding 232
    encoding 54
    encoding 15
    encoding 155
    encoding 132
    encoding 154
    encoding 200
    encoding 11
    encoding 101
    encoding 138
    encoding 69
    encoding 29
    encoding 42
    encoding 112
    encoding 87
    encoding 73
    encoding 64
    encoding 3
    encoding 113
    encoding 34
    encoding 133
    encoding 162
    encoding 59
    encoding 146
    encoding 250
    encoding 139
    encoding 8
    encoding 160
    encoding 258
    encoding 28
    encoding 22
    encoding 203
    encoding 184
    encoding 85
    encoding 52
    encoding 35
    encoding 16
    encoding 183
    encoding 171
    encoding 187
    encoding 71
    encoding 188
    encoding 98
    encoding 223
    encoding 195
    encoding 47
    encoding 107
    encoding 214
    encoding 179
    encoding 202
    encoding 96
    encoding 248
    encoding 221
    enco

In [280]:
jan_2017 = one_hot(jan_2017, 'PUBorough')
print('done encoding PUBorough')

    encoding Queens
    encoding EWR
    encoding Brooklyn
    encoding Staten Island
    encoding Manhattan
    encoding Bronx
done encoding PUBorough


In [282]:
jan_2017 = one_hot(jan_2017, 'PUServiceZone')
print('done encoding PUServiceZone')

    encoding EWR
    encoding Yellow Zone
    encoding Airports
    encoding Boro Zone
done encoding PUServiceZone


In [286]:
jan_2017 = one_hot(jan_2017, 'PUNeighbor')
print('done encoding PUNeighbor')

In [ ]:
jan_2017 = one_hot(jan_2017, 'PU_DOW')
print('done encoding PU_DOW')

In [183]:
jan_2017 = one_hot(jan_2017, 'PUEvents')
print('done encoding PUEvents')

done encoding PUEvents


In [184]:
jan_2017 = one_hot(jan_2017, 'PUConditions')
print('done encoding PUConditions')

done encoding PUConditions


In [185]:
jan_2017 = one_hot(jan_2017, 'PUPeriod')
print('done encoding PUPeriod')

done encoding PUPeriod


In [186]:
jan_2017 = one_hot(jan_2017, 'PUDay')
print('done encoding PUDay')

done encoding PUDay


In [ ]:
jan_2017.printSchema()

In [188]:
print(len(jan_2017.schema.names))

1276


In [ ]:
def plot_points(X, labels):
        plt.scatter(X[:,0],X[:,1])
        for i, txt in enumerate(labels):
                plt.annotate(txt, (X[i,0],X[i,1]))
        plt.show()

In [ ]:
## PCA!!!
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Select Labels
labels = jan_2017['PUZone']

# Standardize everything
sc = StandardScaler()
jan_2017_std= sc.fit_transform(jan_2017)

# Compute PCA with 2 Principal Components
pca = PCA(n_components=2)
pca.fit(jan_2017_std)
jan_2017_PCA = pca.transform(jan_2017_std)
